In [1]:
import os
import pandas
import json
import numpy as np
import pandas as pd

import networkx as nx
from matplotlib import pyplot as plt
from collections import Counter

In [22]:
# import features df
video_info = pd.read_csv('../../data/derived_data/analysis/video_info.csv')

# import channel leanings and merge into features
channel_leanings = pd.read_csv('../../data/derived_data/analysis/channel_classification.csv')
video_info = video_info.merge(channel_leanings, on='channel', how='left')

# import channel information
channel_info = pd.read_csv('../../data/derived_data/analysis/channel_info.csv')

In [29]:
# create weighted digraph from multi-edge adjacency list
G = nx.DiGraph()
edges = []
with open(os.path.join('../../data/derived_data/analysis/channel_adj.txt'), 'r') as f:
    for line in f:
        line_tuple = line.rstrip().split(',')
        parent_node = line_tuple[0]
        children_dict = dict(Counter(line_tuple[1:]))
        edge_tuple = [[parent_node, child, children_dict[child]] for child in children_dict]
        edges += edge_tuple

G.add_weighted_edges_from(edges)

# add in channel leanings
leaning = dict(zip(channel_leanings.channel, channel_leanings.leaning))
nx.set_node_attributes(G, name='leaning', values=leaning)

# save graph as graphml object
nx.write_graphml(G, '../../data/derived_data/analysis/channel_graph.graphml')

In [32]:
channel_leanings

,channel,leaning
0,bbc,L
1,bloomberg,R
2,business insider,C
3,c-span,L
4,cbs,R
5,chicago tribune,C
6,cnbc,C
7,cnn,L
8,daily mail,R
9,financial times,R
